<a href="https://colab.research.google.com/github/AdamRolander/RAG-Experiments/blob/main/Final_RapidFire_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retrieval-First RAG Optimization: A Systematic Study on QASPER

**Dataset:** QASPER (Question Answering on Scientific Papers)

## Abstract
This notebook implements a systematic, two-phase optimization strategy to maximize retrieval performance (MRR, Recall@K, NDCG@K) on the complex QASPER dataset. Unlike standard RAG implementations, we decouple the pipeline into isolated experimental variables:


1.  **Phase 1 (Granularity):** Comparing Fixed-size vs. Semantic/Structural chunking & overlap strategies.
2.  **Phase 2 (Representation):** Benchmarking 2021 baselines against SOTA 2024 embedding models (BGE-M3, Nomic, etc.).

**Framework:** RapidFire AI is utilized for hyperparallel experiment execution and grid-search capabilities.

## 1. Environment Setup & Shared Utilities

We initialize the environment by installing the RapidFire framework and necessary NLP dependencies. To ensure code modularity, we define our **Evaluation Metrics** (MRR, Recall@K, NDCG) and **Data Processing** functions globally here. These will be reused across all three experimental phases to ensure consistent measurement.

In [ ]:
import os
import sys
import subprocess
import time
import gc
import json
import math
import random
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List, Dict, Any

os.system("pkill -f pyright")
os.system("pkill -f vllm")
os.system("ray stop --force")

!pip uninstall fsspec -y -q
!pip install "fsspec==2025.3.0" # Pinning to a stable 2024 version is safer for now
!pip install "huggingface-hub<1.0.0"
!pip install sentence-transformers rank_bm25 nltk -q
!apt install libomp-dev
!pip install --upgrade faiss-gpu-cu12
!pip install "datasets==2.21.0" "huggingface-hub<1.0.0"
!pip install einops -q

# Check if libraries are present; if not, run setup
try:
    import rapidfireai
    import datasets
    print(f"✓ RapidFire {rapidfireai.__version__} already installed.")
except ImportError:
    %pip install rapidfireai==0.14.0  # Takes 1 min
    !rapidfireai init --evals # Takes 1 min
    print("⚠️  PLEASE RESTART RUNTIME NOW (Runtime > Restart Session) to load new versions.")

# --- 2. Global Imports & Configuration ---
import nltk
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)

# RapidFire & LangChain Imports
try:
    from rapidfireai import Experiment
    from rapidfireai.automl import RFLangChainRagSpec, RFvLLMModelConfig, RFGridSearch
    from langchain_community.document_loaders import DirectoryLoader, JSONLoader
    from langchain_huggingface import HuggingFaceEmbeddings
    from langchain_text_splitters import TextSplitter
except ImportError:
    print("❌ Libraries not found. Did you restart the runtime after installation?")

# Set Random Seeds for Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

print(f"✓ Environment Ready. Seed set to {SEED}.")

# --- 3. Shared Metric Functions (Used in Phase 1, 2, & 3) ---

def compute_metrics_fn(batch: Dict[str, List]) -> Dict[str, Dict[str, Any]]:
    """Compute retrieval metrics (MRR, Recall, NDCG) for a batch."""
    total_queries = len(batch["query"])
    mrrs, recalls_10, ndcgs_10 = [], [], []

    # Handle variable key names for ground truth
    gt_key = "ground_truth_documents" if "ground_truth_documents" in batch else "ground_truth_id"

    for retrieved, gt in zip(batch["retrieved_documents"], batch[gt_key]):
        # Normalization: Ensure GT is a set of strings
        if isinstance(gt, list):
            relevant = set(str(g) for g in gt)
        else:
            relevant = {str(gt)}

        # MRR (Mean Reciprocal Rank)
        mrr = 0.0
        for i, doc_id in enumerate(retrieved):
            if doc_id in relevant:
                mrr = 1.0 / (i + 1)
                break
        mrrs.append(mrr)

        # Recall@10
        retrieved_set_10 = set(retrieved[:10])
        recalls_10.append(len(retrieved_set_10 & relevant) / len(relevant) if relevant else 0.0)

        # NDCG@10
        dcg = sum(1.0 / math.log2(i + 2) for i, d in enumerate(retrieved[:10]) if d in relevant)
        idcg = sum(1.0 / math.log2(i + 2) for i in range(min(10, len(relevant))))
        ndcgs_10.append(dcg / idcg if idcg > 0 else 0.0)

    return {
        "Total": {"value": total_queries},
        "MRR": {"value": sum(mrrs) / total_queries},
        "Recall@10": {"value": sum(recalls_10) / total_queries},
        "NDCG@10": {"value": sum(ndcgs_10) / total_queries},
    }

def accumulate_metrics_fn(aggregated: Dict[str, List]) -> Dict[str, Dict[str, Any]]:
    """Aggregate batch metrics into a final global score."""
    # Calculate total queries across all batches
    total = sum(m["value"] for m in aggregated["Total"])
    if total == 0: return {}

    # Weighted average of batch metrics
    metrics = ["MRR", "Recall@10", "NDCG@10"]
    return {
        metric: {
            "value": sum(x["value"] * n["value"] for x, n in zip(aggregated[metric], aggregated["Total"])) / total,
            "is_algebraic": True
        }
        for metric in metrics
    }

print("✓ Shared metric functions defined.")

## 2. Dataset Preparation (QASPER)

QASPER contains 1,585 NLP papers with 5,049 expert-annotated questions. The data preprocessing pipeline below performs three critical steps:
1.  **Corpus Flattening:** Converts nested JSON (sections/paragraphs) into a flat list of documents.
2.  **Evidence Extraction:** Identifies the "Gold Standard" evidence spans for evaluation.
3.  **RapidFire Serialization:** Exports the corpus to `.jsonl` and relevance judgments (qrels) to `.tsv` for efficient ingestion.

### I. Load Dataset

In [ ]:
from datasets import load_dataset

print("="*70)
print("LOADING QASPER DATASET")
print("="*70)

# Add trust_remote_code=True
qasper = load_dataset("allenai/qasper", trust_remote_code=True)

print(f"\n✓ Dataset loaded:")
print(f"  Train: {len(qasper['train'])} papers")
print(f"  Validation: {len(qasper['validation'])} papers")

# Combine train and validation for our corpus
all_papers = list(qasper['train']) + list(qasper['validation'])
print(f"  Total: {len(all_papers)} papers")

# Explore structure
sample_paper = all_papers[0]
print(f"\n✓ Paper structure:")
print(f"  Keys: {list(sample_paper.keys())}")
print(f"  Title: {sample_paper['title'][:80]}...")
print(f"  Sections: {len(sample_paper['full_text']['section_name'])} sections")
print(f"  Questions: {len(sample_paper['qas']['question'])} questions")

### II. Preprocess QASPER into Corpus + Eval Set

Convert QASPER's nested structure into:
1. **Corpus**: Flat documents with full paper text
2. **Eval Set**: Questions with ground truth paper IDs and evidence spans

In [ ]:
from typing import Dict, List
import tqdm
ListType = List

print("\n" + "="*70)
print("PREPROCESSING QASPER")
print("="*70)

def extract_full_text(paper: Dict) -> str:
    """Combine all sections of a paper into full text."""
    sections = paper['full_text']
    full_text_parts = []

    for section_name, paragraphs in zip(sections['section_name'], sections['paragraphs']):
        if section_name:
            full_text_parts.append(f"\n## {section_name}\n")
        full_text_parts.extend(paragraphs)

    return '\n'.join(full_text_parts)


def extract_evidence_text(paper: Dict, evidence_list: ListType) -> str:
    """Extract evidence text from paper given evidence annotations."""
    evidence_texts = []
    for evidence in evidence_list:
        if evidence.get('highlighted_evidence'):
            evidence_texts.extend(evidence['highlighted_evidence'])
    return ' '.join(evidence_texts)


# Build corpus
corpus = []
paper_id_to_idx = {}

for idx, paper in enumerate(tqdm.tqdm(all_papers, desc="Building corpus")):
    paper_id = paper['id']
    full_text = extract_full_text(paper)

    # Skip papers with very little text
    word_count = len(full_text.split())
    if word_count < 100:
        continue

    corpus.append({
        'id': paper_id,
        'title': paper['title'],
        'abstract': paper['abstract'],
        'full_text': full_text,
        'word_count': word_count
    })
    paper_id_to_idx[paper_id] = len(corpus) - 1

print(f"\n✓ Corpus built: {len(corpus)} papers")

# Compute corpus statistics
word_counts = [doc['word_count'] for doc in corpus]
print(f"\n  Document length statistics:")
print(f"    Mean: {np.mean(word_counts):.0f} words")
print(f"    Median: {np.median(word_counts):.0f} words")
print(f"    Min: {np.min(word_counts)} words")
print(f"    Max: {np.max(word_counts)} words")

# Build evaluation set
eval_set = []
papers_with_questions = set()

for paper in tqdm.tqdm(all_papers, desc="Building eval set"):
    paper_id = paper['id']

    # Skip if paper not in corpus
    if paper_id not in paper_id_to_idx:
        continue

    qas = paper['qas']

    for q_idx, question in enumerate(qas['question']):
        # Get answers and evidence
        answers = qas['answers'][q_idx]

        # Extract evidence from all annotators
        all_evidence = []
        for answer in answers['answer']:
            if answer.get('highlighted_evidence'):
                all_evidence.extend(answer['highlighted_evidence'])

        # Only include questions with evidence (answerable from the paper)
        if not all_evidence:
            continue

        eval_set.append({
            'query': question,
            'query_id': f"{paper_id}_q{q_idx}",
            'ground_truth_id': paper_id,
            'ground_truth_title': paper['title'],
            'evidence_texts': all_evidence[:5],  # Keep top 5 evidence spans
        })
        papers_with_questions.add(paper_id)

print(f"\n✓ Eval set built: {len(eval_set)} questions")
print(f"  From {len(papers_with_questions)} unique papers")

# Sample questions
print(f"\n  Sample questions:")
for i, q in enumerate(random.sample(eval_set, min(3, len(eval_set)))):
    print(f"  {i+1}. {q['query'][:80]}...")
    print(f"     Paper: {q['ground_truth_id']}")

### III. Save Preprocessed Data for RapidFire

Export corpus and eval set in formats compatible with RapidFire's data loaders.

In [ ]:
print("\n" + "="*70)
print("EXPORTING DATA FOR RAPIDFIRE")
print("="*70)

# Create data directory
data_dir = Path("qasper_data")
data_dir.mkdir(exist_ok=True)

# Save corpus as JSONL (for LangChain JSONLoader)
corpus_path = data_dir / "corpus.jsonl"
with open(corpus_path, 'w') as f:
    for doc in corpus:
        # Format for JSONLoader
        record = {
            '_id': doc['id'],
            'text': doc['full_text'],
            'title': doc['title'],
            'abstract': doc['abstract']
        }
        f.write(json.dumps(record) + '\n')

print(f"✓ Corpus saved: {corpus_path}")

# Save eval set
eval_path = data_dir / "eval_set.json"
with open(eval_path, 'w') as f:
    json.dump(eval_set, f, indent=2)

print(f"✓ Eval set saved: {eval_path}")

# Create qrels (relevance judgments) for RapidFire compatibility
qrels_data = []
for item in eval_set:
    qrels_data.append({
        'query_id': item['query_id'],
        'corpus_id': item['ground_truth_id'],
        'relevance': 1
    })

qrels_df = pd.DataFrame(qrels_data)
qrels_path = data_dir / "qrels.tsv"
qrels_df.to_csv(qrels_path, sep='\t', index=False)

print(f"✓ Qrels saved: {qrels_path}")

# Save statistics
stats = {
    'corpus_size': len(corpus),
    'eval_queries': len(eval_set),
    'unique_papers_with_questions': len(papers_with_questions),
    'avg_document_length': float(np.mean(word_counts)),
    'median_document_length': float(np.median(word_counts)),
}

with open(data_dir / "dataset_stats.json", 'w') as f:
    json.dump(stats, f, indent=2)

print(f"\n✓ Dataset statistics:")
for k, v in stats.items():
    print(f"  {k}: {v}")

### IV. Load Data into DataFrames

Prepare data structures for experimentation.

In [ ]:
print("\n" + "="*70)
print("LOADING DATA FOR EXPERIMENTS")
print("="*70)

# Load corpus
corpus_df = pd.read_json(corpus_path, lines=True)
corpus_df = corpus_df.rename(columns={'_id': 'id'})
print(f"✓ Corpus loaded: {len(corpus_df)} documents")

# Load eval set
eval_df = pd.DataFrame(eval_set)
print(f"✓ Eval set loaded: {len(eval_df)} queries")

# Load qrels
qrels = pd.read_csv(qrels_path, sep='\t')
qrels['corpus_id'] = qrels['corpus_id'].astype(str)
print(f"✓ Qrels loaded: {len(qrels)} relevance judgments")
print(f"Qrels corpus_id type: {qrels['corpus_id'].dtype}")
print(f"Sample: {qrels['corpus_id'].iloc[0]} (type: {type(qrels['corpus_id'].iloc[0])})")

# Create document lookup
doc_lookup = {row['id']: row['text'] for _, row in corpus_df.iterrows()}

print(f"\n✓ Data ready for experimentation")

## Phase 1: Granularity Optimization (Chunking)

**Hypothesis:** Scientific literature contains complex logical boundaries. Semantic chunking (based on topic shifts) will yield higher retrieval precision than arbitrary fixed-window chunking.

**Methodology:**
We implement a factorial design comparing:
* **Strategies:** Fixed Token (256/512), Structural (Section-based), and Semantic (BM25-similarity).
* **Overlap:** None vs. Fixed (15%) vs. Semantic (context-aware extension).

The winning configuration from this phase will be "locked in" for Phases 2 and 3.

### I. Initialize Experiment

In [ ]:
print("\n" + "="*70)
print("INITIALIZING RAPIDFIRE EXPERIMENT")
print("="*70)

experiment = Experiment(
    experiment_name="qasper-phase1-chunking",
    mode="evals"
)

print("✓ Experiment initialized: qasper-phase1-chunking")

### II. Define Custom Chunking & Overlap Strategies

**Chunking Schemes:**
1. Fixed-size (256, 512 tokens)
2. Structural (paragraph/section boundaries)
3. Semantic (BM25-guided boundary detection)

**Overlap Strategies:**
1. No overlap (0%)
2. Fixed overlap (15%)
3. Semantic overlap (BM25-guided context extension)

Full factorial: 4 chunkers × 3 overlaps = 12 configurations

In [ ]:
from langchain_text_splitters import TextSplitter
from rank_bm25 import BM25Okapi
from typing import List as ListType, Iterable
import re
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize

# -----------------------------------------------------------------------------
# 1. BASE CHUNKING STRATEGIES
# -----------------------------------------------------------------------------

class FixedTokenSplitter(TextSplitter):
    """Split text into fixed-size token chunks."""

    def __init__(self, chunk_size: int = 256, chunk_overlap: int = 0, **kwargs):
        super().__init__(chunk_size=chunk_size, chunk_overlap=chunk_overlap, **kwargs)
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self._name = f"fixed_{chunk_size}tok_overlap{chunk_overlap}"

    def split_text(self, text: str) -> ListType[str]:
        # Import inside method for Ray worker safety
        from nltk.tokenize import word_tokenize

        tokens = word_tokenize(text)
        chunks = []
        step = max(1, self.chunk_size - self.chunk_overlap)

        for start in range(0, len(tokens), step):
            end = min(start + self.chunk_size, len(tokens))
            chunk_tokens = tokens[start:end]
            if chunk_tokens:
                chunks.append(' '.join(chunk_tokens))
            if end >= len(tokens):
                break
        return chunks


class StructuralSplitter(TextSplitter):
    """Split text at paragraph and section boundaries."""

    def __init__(self, min_chunk_size: int = 50, max_chunk_size: int = 500,
                 overlap_sentences: int = 0, **kwargs):
        super().__init__(chunk_size=max_chunk_size, chunk_overlap=overlap_sentences, **kwargs)
        self.min_chunk_size = min_chunk_size
        self.max_chunk_size = max_chunk_size
        self.overlap_sentences = overlap_sentences
        self._name = f"structural_overlap{overlap_sentences}sent"

    def split_text(self, text: str) -> ListType[str]:
        # Import inside method for Ray worker safety
        import re
        from nltk.tokenize import sent_tokenize, word_tokenize

        # Split on section headers (## Header) and double newlines
        sections = re.split(r'\n##\s+|\n\n+', text)
        sections = [s.strip() for s in sections if s.strip()]

        chunks = []
        current_chunk = []
        current_length = 0
        previous_sentences = []

        for section in sections:
            sentences = sent_tokenize(section)

            for sent in sentences:
                sent_tokens = len(word_tokenize(sent))

                if current_length + sent_tokens > self.max_chunk_size and current_chunk:
                    chunk_text = ' '.join(current_chunk)
                    chunks.append(chunk_text)

                    if self.overlap_sentences > 0:
                        previous_sentences = current_chunk[-self.overlap_sentences:]
                        current_chunk = previous_sentences.copy()
                        current_length = sum(len(word_tokenize(s)) for s in current_chunk)
                    else:
                        current_chunk = []
                        current_length = 0

                current_chunk.append(sent)
                current_length += sent_tokens

        if current_chunk and current_length >= self.min_chunk_size:
            chunks.append(' '.join(current_chunk))
        elif current_chunk and chunks:
            chunks[-1] = chunks[-1] + ' ' + ' '.join(current_chunk)
        elif current_chunk:
            chunks.append(' '.join(current_chunk))

        return chunks if chunks else [text]


class SemanticBM25Splitter(TextSplitter):
    """Split text at semantic boundaries using BM25 similarity."""

    def __init__(self, similarity_threshold: float = 0.3,
                 min_chunk_sentences: int = 3, **kwargs):
        super().__init__(chunk_size=500, chunk_overlap=0, **kwargs)
        self.similarity_threshold = similarity_threshold
        self.min_chunk_sentences = min_chunk_sentences
        self._name = f"semantic_bm25_thresh{similarity_threshold}"

    def split_text(self, text: str) -> ListType[str]:
        # Import inside method for Ray worker safety
        from nltk.tokenize import sent_tokenize, word_tokenize
        from rank_bm25 import BM25Okapi

        sentences = sent_tokenize(text)

        if len(sentences) <= self.min_chunk_sentences:
            return [text]

        tokenized_sents = [word_tokenize(s.lower()) for s in sentences]
        tokenized_sents = [t if t else [''] for t in tokenized_sents]

        try:
            bm25 = BM25Okapi(tokenized_sents)
        except:
            return [text]

        boundaries = [0]
        for i in range(self.min_chunk_sentences, len(sentences) - 1):
            prev_context = ' '.join(sentences[max(0, i-3):i])
            prev_tokens = word_tokenize(prev_context.lower())

            if not prev_tokens: continue

            scores = bm25.get_scores(prev_tokens)
            max_score = max(scores) if max(scores) > 0 else 1

            if scores[i] < self.similarity_threshold * max_score:
                if i - boundaries[-1] >= self.min_chunk_sentences:
                    boundaries.append(i)

        boundaries.append(len(sentences))

        chunks = []
        for i in range(len(boundaries) - 1):
            chunk_sents = sentences[boundaries[i]:boundaries[i+1]]
            if chunk_sents:
                chunks.append(' '.join(chunk_sents))

        return chunks if chunks else [text]


# -----------------------------------------------------------------------------
# 2. OVERLAP WRAPPERS
# -----------------------------------------------------------------------------

class FixedSentenceOverlapWrapper(TextSplitter):
    """Wraps a base splitter and adds fixed sentence overlap."""

    def __init__(self, base_splitter: TextSplitter, overlap_sentences: int = 2, **kwargs):
        super().__init__(chunk_size=base_splitter._chunk_size,
                         chunk_overlap=overlap_sentences * 20, **kwargs)
        self.base_splitter = base_splitter
        self.overlap_sentences = overlap_sentences
        self._name = f"{getattr(base_splitter, '_name', 'base')}_fixedoverlap{overlap_sentences}"

    def split_text(self, text: str) -> ListType[str]:
        # Import inside method for Ray worker safety
        from nltk.tokenize import sent_tokenize

        base_chunks = self.base_splitter.split_text(text)
        if len(base_chunks) <= 1 or self.overlap_sentences == 0:
            return base_chunks

        extended_chunks = []
        for i, chunk in enumerate(base_chunks):
            parts = []
            if i > 0:
                prev_sents = sent_tokenize(base_chunks[i-1])
                parts.extend(prev_sents[-self.overlap_sentences:])
            parts.append(chunk)
            extended_chunks.append(' '.join(parts))
        return extended_chunks


class SemanticOverlapWrapper(TextSplitter):
    """Wraps base splitter; adds overlap based on BM25 semantic similarity of boundary sentences."""

    def __init__(self, base_splitter: TextSplitter, similarity_threshold: float = 0.2, **kwargs):
        super().__init__(chunk_size=base_splitter._chunk_size,
                         chunk_overlap=50, **kwargs)
        self.base_splitter = base_splitter
        self.similarity_threshold = similarity_threshold
        base_name = getattr(base_splitter, '_name', 'base')
        self._name = f"{base_name}_semoverlap"

    def split_text(self, text: str) -> ListType[str]:
        # Import inside method for Ray worker safety
        from nltk.tokenize import sent_tokenize, word_tokenize
        from rank_bm25 import BM25Okapi

        base_chunks = self.base_splitter.split_text(text)
        if len(base_chunks) <= 1:
            return base_chunks

        boundary_sents = []
        for chunk in base_chunks:
            sents = sent_tokenize(chunk)
            boundary_sents.append(sents[0] if sents else "")
            boundary_sents.append(sents[-1] if sents else "")

        tokenized = [word_tokenize(s.lower()) for s in boundary_sents]
        tokenized = [t if t else [''] for t in tokenized]

        try:
            bm25 = BM25Okapi(tokenized)
        except:
            return base_chunks

        extended_chunks = []
        for i, chunk in enumerate(base_chunks):
            parts = []

            if i > 0:
                prev_last_idx = (i - 1) * 2 + 1
                curr_first_idx = i * 2

                query = tokenized[curr_first_idx]
                if query and query != ['']:
                    scores = bm25.get_scores(query)
                    max_score = max(scores) if max(scores) > 0 else 1

                    if scores[prev_last_idx] > self.similarity_threshold * max_score:
                        prev_sents = sent_tokenize(base_chunks[i-1])
                        if prev_sents:
                            parts.append(prev_sents[-1])

            parts.append(chunk)

            if i < len(base_chunks) - 1:
                curr_last_idx = i * 2 + 1
                next_first_idx = (i + 1) * 2

                query = tokenized[curr_last_idx]
                if query and query != ['']:
                    scores = bm25.get_scores(query)
                    max_score = max(scores) if max(scores) > 0 else 1

                    if scores[next_first_idx] > self.similarity_threshold * max_score:
                        next_sents = sent_tokenize(base_chunks[i+1])
                        if next_sents:
                            parts.append(next_sents[0])

            extended_chunks.append(' '.join(parts))

        return extended_chunks


# -----------------------------------------------------------------------------
# 3. CREATE ALL 12 CONFIGURATIONS
# -----------------------------------------------------------------------------

print("="*70)
print("CONFIGURING CHUNKING EXPERIMENTS")
print("="*70)

# Base chunkers (no overlap)
fixed_256_base = FixedTokenSplitter(chunk_size=256, chunk_overlap=0)
fixed_512_base = FixedTokenSplitter(chunk_size=512, chunk_overlap=0)
structural_base = StructuralSplitter(min_chunk_size=50, max_chunk_size=500, overlap_sentences=0)
semantic_base = SemanticBM25Splitter(similarity_threshold=0.3, min_chunk_sentences=3)

# Fixed overlap versions (15% overlap)
fixed_256_fixedoverlap = FixedTokenSplitter(chunk_size=256, chunk_overlap=38)
fixed_512_fixedoverlap = FixedTokenSplitter(chunk_size=512, chunk_overlap=77)
structural_fixedoverlap = StructuralSplitter(min_chunk_size=50, max_chunk_size=500, overlap_sentences=2)
semantic_fixedoverlap = FixedSentenceOverlapWrapper(
    SemanticBM25Splitter(similarity_threshold=0.3, min_chunk_sentences=3),
    overlap_sentences=2
)

# Semantic overlap versions (Standardized threshold 0.2)
fixed_256_semoverlap = SemanticOverlapWrapper(FixedTokenSplitter(chunk_size=256, chunk_overlap=0), similarity_threshold=0.2)
fixed_512_semoverlap = SemanticOverlapWrapper(FixedTokenSplitter(chunk_size=512, chunk_overlap=0), similarity_threshold=0.2)
structural_semoverlap = SemanticOverlapWrapper(StructuralSplitter(min_chunk_size=50, max_chunk_size=500), similarity_threshold=0.2)
semantic_semoverlap = SemanticOverlapWrapper(SemanticBM25Splitter(similarity_threshold=0.3), similarity_threshold=0.2)

ALL_SPLITTERS = [
    # No overlap
    ("fixed_256_no_overlap", fixed_256_base),
    ("fixed_512_no_overlap", fixed_512_base),
    ("structural_no_overlap", structural_base),
    ("semantic_no_overlap", semantic_base),
    # Fixed 15% overlap
    ("fixed_256_fixed_overlap", fixed_256_fixedoverlap),
    ("fixed_512_fixed_overlap", fixed_512_fixedoverlap),
    ("structural_fixed_overlap", structural_fixedoverlap),
    ("semantic_fixed_overlap", semantic_fixedoverlap),
    # Semantic BM25 overlap
    ("fixed_256_semantic_overlap", fixed_256_semoverlap),
    ("fixed_512_semantic_overlap", fixed_512_semoverlap),
    ("structural_semantic_overlap", structural_semoverlap),
    ("semantic_semantic_overlap", semantic_semoverlap),
]

print(f"\n✓ Created {len(ALL_SPLITTERS)} chunking configurations.")

### III. Execution Logic (Pre/Post Processing)

We define the specific runtime logic required for the RapidFire evaluator:

1.  **`preprocess_fn`**: The "engine" of the experiment. It accepts a batch of queries, executes the retrieval using the specific chunking configuration being tested, and formats the results.
2.  **`postprocess_fn`**: The "judge". It looks up the Ground Truth document IDs (from the `qrels` dataframe) and attaches them to the results so we can calculate accuracy.

*Note: The actual scoring logic (`compute_metrics_fn`) is inherited from the Global Setup step to ensure consistency across all phases.*

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
from typing import Dict, List, Any

def preprocess_fn(
    batch: Dict[str, List],
    rag: RFLangChainRagSpec,
    prompt_manager=None
) -> Dict[str, List]:
    """
    Phase 1 Retrieval Execution:
    1. Runs the RAG retrieval using the current chunking config.
    2. Extracts retrieved Document IDs.
    3. Serializes context for the LLM (minimal prompt).
    """

    # 1. Perform batched retrieval
    # serialize=False lets us access the raw Document objects to get metadata
    all_context = rag.get_context(batch_queries=batch["query"], serialize=False)

    # 2. Extract retrieved document IDs (for metric calculation)
    retrieved_documents = [
        [doc.metadata.get("doc_id", "") for doc in docs]
        for docs in all_context
    ]

    # 3. Serialize context for the generator
    serialized_context = rag.serialize_documents(all_context)

    # 4. Format prompts (Minimal structure, we focus on retrieval metrics here)
    prompts = [
        [
            {"role": "system", "content": "Answer briefly."},
            {"role": "user", "content": f"Context: {ctx[:500]}\n\nQuestion: {q}\n\nAnswer:"}
        ]
        for q, ctx in zip(batch["query"], serialized_context)
    ]

    return {
        "prompts": prompts,
        "retrieved_documents": retrieved_documents,
        **batch,
    }


def postprocess_fn(batch: Dict[str, List]) -> Dict[str, List]:
    """
    Phase 1 Ground Truth Attachment:
    Looks up the correct 'corpus_id' for each query in the global 'qrels' DataFrame.
    """

    # Check if we have a match in the QRELS (Relevance Judgments) file
    batch["ground_truth_documents"] = [
        [str(qrels[qrels["query_id"] == qid]["corpus_id"].iloc[0])]
        if len(qrels[qrels["query_id"] == qid]) > 0 else []
        for qid in batch["query_id"]
    ]

    return batch

print("✓ Phase 1 Execution Logic defined (Pre/Post processors).")

### IV. Sequential & Mounted Experiment Execution

In [ ]:
import gc
import torch
from google.colab import drive

# Mount Google Drive for persistent storage
drive.mount('/content/drive')
results_dir = '/content/drive/MyDrive/RapidFire_QASPER_Results/phase1'
os.makedirs(results_dir, exist_ok=True)
print(f"✓ Results will be saved to: {results_dir}")

In [ ]:
from rapidfireai.automl import RFvLLMModelConfig

# =============================================================================
# SERIAL EXECUTION - ONE CONFIG AT A TIME (Colab-safe)
# =============================================================================

batch_size = 32

# Prepare eval dataset (same as before)
from datasets import Dataset
sample_fraction = 0.1
sample_size = int(len(eval_df) * sample_fraction)
eval_dataset = Dataset.from_pandas(eval_df.sample(n=sample_size, random_state=42))
print(f"✓ Eval dataset: {len(eval_dataset)} queries ({sample_fraction*100:.0f}% sample)")

# Store all results
all_results = []

print("\n" + "="*70)
print("RUNNING SERIAL EXPERIMENTS (12 configs)")
print("="*70)

for config_idx, (config_name, splitter) in enumerate(ALL_SPLITTERS):

    # import ray
    # if ray.is_initialized():
    #     ray.shutdown()
    #     import time
    #     time.sleep(5)

    # Check if already completed (resume capability)
    save_path = f"{results_dir}/{config_name}_results.csv"
    if os.path.exists(save_path):
        print(f"\n⏭️  [{config_idx+1}/12] Skipping {config_name} (already done)")
        all_results.append(pd.read_csv(save_path))
        continue

    print(f"\n{'='*60}")
    print(f"🚀 [{config_idx+1}/12] Running: {config_name}")
    print(f"{'='*60}")

    # Create RAG config for THIS splitter only (not List)
    rag_config = RFLangChainRagSpec(
        document_loader=DirectoryLoader(
            path=str(data_dir),
            glob="corpus.jsonl",
            loader_cls=JSONLoader,
            loader_kwargs={
                "jq_schema": ".",
                "content_key": "text",
                "metadata_func": lambda record, metadata: {
                    "doc_id": record.get("_id"),
                    "title": record.get("title", "")
                },
                "json_lines": True,
                "text_content": False,
            },
        ),
        text_splitter=splitter,  # SINGLE splitter, not List
        embedding_cls=HuggingFaceEmbeddings,
        embedding_kwargs={
            "model_name": "sentence-transformers/all-MiniLM-L6-v2",
            "model_kwargs": {"device": "cuda:0"},
            "encode_kwargs": {"normalize_embeddings": True, "batch_size": batch_size},
        },
        vector_store=None,
        search_type="similarity",
        search_kwargs={"k": 20},
        reranker_cls=None,
        reranker_kwargs=None,
        enable_gpu_search=True,
    )

    # vLLM config
    vllm_config = RFvLLMModelConfig(
        model_config={
            "model": "Qwen/Qwen2.5-0.5B-Instruct",
            "dtype": "half",
            "gpu_memory_utilization": 0.2,
            "tensor_parallel_size": 1,
            "distributed_executor_backend": "mp",
            "enable_chunked_prefill": False,
            "enable_prefix_caching": False,
            "max_model_len": 1024,
            "disable_log_stats": True,
            "enforce_eager": True,
            "disable_custom_all_reduce": True,
        },
        sampling_params={
            "temperature": 0.1,
            "top_p": 0.95,
            "max_tokens": 16,
        },
        rag=rag_config,
        prompt_manager=None,
    )

    # Config set
    config_set = {
        "vllm_config": vllm_config,
        "batch_size": batch_size,
        "preprocess_fn": preprocess_fn,
        "postprocess_fn": postprocess_fn,
        "compute_metrics_fn": compute_metrics_fn,
        "accumulate_metrics_fn": accumulate_metrics_fn,
        "online_strategy_kwargs": {
            "strategy_name": "normal",
            "confidence_level": 0.95,
            "use_fpc": True,
        },
    }

    config_group = RFGridSearch(config_set)

    # Fresh experiment per config
    exp = Experiment(
        experiment_name=f"qasper_{config_name}",
        mode="evals"
    )

    try:
        # Run evaluation
        results = exp.run_evals(
            config_group=config_group,
            dataset=eval_dataset,
            num_actors=1,
            num_shards=4,
            seed=42,
        )

        # Extract metrics
        for run_id, (config, metrics_dict) in results.items():
            row = {
                'config_name': config_name,
                'run_id': run_id,
                'chunk_size': splitter._chunk_size,
                'chunk_overlap': splitter._chunk_overlap,
            }
            for k, v in metrics_dict.items():
                if isinstance(v, dict) and 'value' in v:
                    row[k] = v['value']
                else:
                    row[k] = v

            df = pd.DataFrame([row])
            df.to_csv(save_path, index=False)
            all_results.append(df)

            print(f"   ✅ MRR: {row.get('MRR', 'N/A'):.4f}, Recall@10: {row.get('Recall@10', 'N/A'):.4f}")

        # End experiment
        exp.end()

    except Exception as e:
        print(f"   ❌ Error: {e}")
        # Save error info
        error_df = pd.DataFrame([{'config_name': config_name, 'error': str(e)}])
        error_df.to_csv(f"{results_dir}/{config_name}_ERROR.csv", index=False)

    # Cleanup memory
    del exp, rag_config, vllm_config, config_set, config_group
    gc.collect()
    torch.cuda.empty_cache()
    print(f"   🧹 Memory cleaned")

# Combine all results
print("\n" + "="*70)
print("COMBINING RESULTS")
print("="*70)

results_df = pd.concat(all_results, ignore_index=True)
results_df.to_csv(f"{results_dir}/all_results.csv", index=False)
print(f"✓ Combined results saved: {results_dir}/all_results.csv")
print(f"\n{results_df.to_string(index=False)}")

### V. End Experiment

In [ ]:
try:
    experiment.end()
    print("✓ Phase 1 Experiment finalized and closed.")
except Exception as e:
    print(f"⚠️ Note: Experiment might have already ended or failed: {e}")

# Trigger garbage collection to prep RAM for Phase 2
import gc
gc.collect()

## Phase 2: Representation Optimization (Embeddings)

**Hypothesis:** Newer embedding models (2023-2024 era) optimized for large-scale retrieval tasks (e.g., MTEB leaderboard toppers) will significantly outperform the standard 2021 baselines (`all-MiniLM`, `specter`) on scientific text.

**Experimental Setup:**
Using the **Semantic BM25 Chunker** (Phase 1 Winner), we evaluate a spectrum of embedding models:
* **Baselines:** `all-MiniLM-L6-v2`, `allenai-specter`, `all-mpnet-base-v2`, `multi-qa-mpnet-base`.
* **Modern Dense:** `bge-m3`.
* **Modern Hybrid:** `bge-m3` (Dense + Sparse).

*Note: For the hybrid experiments, we utilize a weighted fusion of dense and sparse vectors to capture both semantic meaning and exact keyword matches.*

### I. Verify Data

In [ ]:
print("="*70)
print("PHASE 2: EMBEDDING MODEL OPTIMIZATION")
print("="*70)

# Verify data is loaded
try:
    print(f"✓ Corpus loaded: {len(corpus_df)} documents")
    print(f"✓ Eval set loaded: {len(eval_df)} queries")
    print(f"✓ Qrels loaded: {len(qrels)} relevance judgments")
except NameError as e:
    print("❌ ERROR: Data not loaded!")
    print("   Please run Phase 1 Cells 1-5 first.")
    print("   (Through 'LOADING DATA FOR EXPERIMENTS')")
    raise e

# Verify random seed consistency
import random
import numpy as np
random.seed(42)
np.random.seed(42)
print(f"✓ Random seeds set to 42 (ensures same eval sample as Phase 1)")


### II. Define Winning Chunking & Overlap Strategy


In [ ]:
from langchain_text_splitters import TextSplitter
from typing import List as ListType

# Instantiate the winning chunker
PHASE1_WINNER_SPLITTER = SemanticBM25Splitter(
    similarity_threshold=0.3,
    min_chunk_sentences=3
)

print("✓ Phase 1 Winner Defined: SemanticBM25Splitter (no overlap)")
print(f"  - Similarity threshold: 0.3")
print(f"  - Min chunk sentences: 3")

# Quick validation
test_text = corpus_df.iloc[0]['text'][:2000]
test_chunks = PHASE1_WINNER_SPLITTER.split_text(test_text)
print(f"  - Validation: {len(test_chunks)} chunks from sample text ✓")


### III. Define Embedding Model Configurations
| Model                 | Era  | Context | Dim  | Prefixes |
|-----------------------|------|---------|------|----------|
| all-MiniLM-L6-v2      | 2021 | 512     | 384  | No       |
| all-mpnet-base-v2     | 2021 | 512     | 768  | No       |
| allenai-specter       | 2020 | 512     | 768  | No       |
| multi-qa-mpnet-base   | 2021 | 512     | 768  | No       |
| bge-m3                | 2024 | 8192    | 1024 | No       |


In [ ]:
EMBEDDING_MODELS = [
    {
        "name": "all-MiniLM-L6-v2",
        "model_name": "sentence-transformers/all-MiniLM-L6-v2",
        "domain": "general",
        "size": "small",
        "dim": 384,
        "max_seq": 512,
        "query_prefix": "",
        "document_prefix": "",
        "trust_remote_code": False,
        "description": "2021 Baseline - General purpose, small & fast"
    },
    {
        "name": "all-mpnet-base-v2",
        "model_name": "sentence-transformers/all-mpnet-base-v2",
        "domain": "general",
        "size": "base",
        "dim": 768,
        "max_seq": 512,
        "query_prefix": "",
        "document_prefix": "",
        "trust_remote_code": False,
        "description": "2021 Baseline - General purpose, larger model"
    },
    {
        "name": "allenai-specter",
        "model_name": "sentence-transformers/allenai-specter",
        "domain": "scientific",
        "size": "base",
        "dim": 768,
        "max_seq": 512,
        "query_prefix": "",
        "document_prefix": "",
        "trust_remote_code": False,
        "description": "2020 Baseline - Scientific papers (citation-trained)"
    },
    {
        "name": "multi-qa-mpnet-base",
        "model_name": "sentence-transformers/multi-qa-mpnet-base-dot-v1",
        "domain": "qa-tuned",
        "size": "base",
        "dim": 768,
        "max_seq": 512,
        "query_prefix": "",
        "document_prefix": "",
        "trust_remote_code": False,
        "description": "2021 Baseline - QA-tuned on 215M pairs"
    },
    {
        "name": "bge-m3",
        "model_name": "BAAI/bge-m3",
        "domain": "retrieval-2024",
        "size": "large",
        "dim": 1024,
        "max_seq": 8192,
        "query_prefix": "",  # BGE-M3 doesn't require prefixes for English
        "document_prefix": "",
        "trust_remote_code": False,
        "description": "2024 SOTA - Multi-granularity, 8K context, dense+sparse+colbert"
    },
]

print("="*70)
print("PHASE 2: EMBEDDING MODEL CONFIGURATIONS")
print("="*70)

# Separate by status
done_models = [m for m in EMBEDDING_MODELS if m['domain'] not in ['retrieval-2024']]
new_models = [m for m in EMBEDDING_MODELS if m['domain'] == 'retrieval-2024']

print(f"\n✓ {len(EMBEDDING_MODELS)} total models configured:")
print(f"  • {len(done_models)} baseline models (2020-2021) - will skip if results exist")
print(f"  • {len(new_models)} new models (2024) - to be run")

print(f"\n{'─'*70}")
print("2024 MODELS TO TEST:")
print(f"{'─'*70}")
for i, model in enumerate(new_models, 1):
    prefix_status = "✓ Requires prefixes" if model['query_prefix'] else "No prefixes needed"
    print(f"\n  {i}. {model['name']}")
    print(f"     Path: {model['model_name']}")
    print(f"     Context: {model['max_seq']} | Dim: {model['dim']} | {prefix_status}")
    print(f"     {model['description']}")

### IV. Initialize Experiment

In [ ]:
from rapidfireai import Experiment

experiment = Experiment(
    experiment_name="qasper-phase2-embeddings",
    mode="evals"
)

print("✓ Experiment initialized: qasper-phase2-embeddings")

# Prepare eval dataset (SAME sampling as Phase 1 via seed=42)
from datasets import Dataset

sample_fraction = 0.1
sample_size = int(len(eval_df) * sample_fraction)
eval_dataset = Dataset.from_pandas(eval_df.sample(n=sample_size, random_state=42))

print(f"✓ Eval dataset: {len(eval_dataset)} queries ({sample_fraction*100:.0f}% sample)")
print(f"  (Same distribution as Phase 1 via random_state=42)")

### V. Experiment Execution (Dense Only)

This cell executes the embedding benchmark for the configured models.

**Configuration:**
* **Models:** We iterate through the `EMBEDDING_MODELS` list defined above.
* **Architecture:** We use the standard `HuggingFaceEmbeddings` class for stability.
* **Resources:** "Safe Mode" is enabled with a reduced batch size (8) and shard count (2) to fit within T4 GPU memory limits.
* **Resume Capability:** The code checks for existing results in Google Drive and skips models that have already completed, allowing for interruption and resumption.

In [ ]:
import gc
import torch
import os
import pandas as pd
from pathlib import Path
from google.colab import drive
from langchain_community.document_loaders import DirectoryLoader, JSONLoader
from langchain_huggingface import HuggingFaceEmbeddings
from rapidfireai import Experiment
from rapidfireai.automl import RFLangChainRagSpec, RFvLLMModelConfig, RFGridSearch

# 1. Setup Resources
results_dir = '/content/drive/MyDrive/RapidFire_QASPER_Results/phase2'
os.makedirs(results_dir, exist_ok=True)
data_dir = Path("qasper_data")

# Ultra-Safe Params for T4 GPU
SAFE_BATCH_SIZE = 8
NUM_SHARDS = 2

print("\n" + "="*70)
print(f"RUNNING PHASE 2 EXECUTION (Safe Mode)")
print("="*70)
print(f"Models to run: {len(EMBEDDING_MODELS)}")
print(f"Batch Size: {SAFE_BATCH_SIZE} | Shards: {NUM_SHARDS}")

all_results = []
gc.collect()

# 2. Execution Loop
for model_idx, model_config in enumerate(EMBEDDING_MODELS):
    model_name = model_config["name"]
    model_path = model_config["model_name"]

    # Path to save individual run results
    save_path = f"{results_dir}/{model_name}_results.csv"

    # --- Check Resume Capability ---
    if os.path.exists(save_path):
        print(f"\n⏭️  [{model_idx+1}/{len(EMBEDDING_MODELS)}] Skipping {model_name} (already done)")
        try:
            all_results.append(pd.read_csv(save_path))
        except: pass
        continue

    print(f"\n{'='*60}")
    print(f"🚀 [{model_idx+1}/{len(EMBEDDING_MODELS)}] Running: {model_name}")
    print(f"   {model_config['description']}")
    print(f"{'='*60}")

    # --- Configure Embeddings (Standard Mode) ---
    # We use standard HuggingFaceEmbeddings which is stable for Ray serialization
    model_kwargs = {"device": "cuda:0"}
    if model_config.get("trust_remote_code"):
        model_kwargs["trust_remote_code"] = True

    rag_config = RFLangChainRagSpec(
        document_loader=DirectoryLoader(
            path=str(data_dir),
            glob="corpus.jsonl",
            loader_cls=JSONLoader,
            loader_kwargs={
                "jq_schema": ".",
                "content_key": "text",
                "metadata_func": lambda record, metadata: {
                    "doc_id": record.get("_id"),
                    "title": record.get("title", "")
                },
                "json_lines": True,
                "text_content": False,
            },
        ),
        # Use the winner from Phase 1
        text_splitter=PHASE1_WINNER_SPLITTER,
        embedding_cls=HuggingFaceEmbeddings,
        embedding_kwargs={
            "model_name": model_path,
            "model_kwargs": model_kwargs,
            "encode_kwargs": {"normalize_embeddings": True, "batch_size": SAFE_BATCH_SIZE},
        },
        vector_store=None,
        search_type="similarity",
        search_kwargs={"k": 20},
        enable_gpu_search=True,
    )

    # --- vLLM Config ---
    vllm_config = RFvLLMModelConfig(
        model_config={
            "model": "Qwen/Qwen2.5-0.5B-Instruct",
            "dtype": "half",
            "gpu_memory_utilization": 0.2,
            "max_model_len": 1024,
            "disable_log_stats": True,
            "enforce_eager": True,
        },
        sampling_params={"temperature": 0.1, "max_tokens": 16},
        rag=rag_config,
    )

    # --- Run Experiment ---
    exp = Experiment(experiment_name=f"qasper_phase2_{model_name}", mode="evals")

    try:
        # Aggressive memory cleanup before run
        gc.collect()
        torch.cuda.empty_cache()

        results = exp.run_evals(
            config_group=RFGridSearch({
                "vllm_config": vllm_config,
                "batch_size": SAFE_BATCH_SIZE,
                "preprocess_fn": preprocess_fn,
                "postprocess_fn": postprocess_fn,
                "compute_metrics_fn": compute_metrics_fn,
                "accumulate_metrics_fn": accumulate_metrics_fn,
            }),
            dataset=eval_dataset,
            num_actors=1,
            num_shards=NUM_SHARDS,
            seed=42,
        )

        # --- Save Results ---
        for run_id, (config, metrics_dict) in results.items():
            row = {
                'phase': 2,
                'embedding_model': model_name,
                'embedding_domain': model_config['domain'],
                'chunking': 'semantic_bm25_no_overlap',
                'run_id': run_id,
            }
            # Extract metrics values safely
            for k, v in metrics_dict.items():
                if isinstance(v, dict) and 'value' in v:
                    row[k] = v['value']
                else:
                    row[k] = v

            df = pd.DataFrame([row])
            df.to_csv(save_path, index=False)
            all_results.append(df)
            print(f"   ✅ MRR: {row.get('MRR', 0):.4f}")

        exp.end()

    except Exception as e:
        print(f"   ❌ Error: {e}")
        # Log error to file so we know what happened
        pd.DataFrame([{'model': model_name, 'error': str(e)}]).to_csv(f"{results_dir}/{model_name}_ERROR.csv")

    # Cleanup after loop
    del exp, rag_config, vllm_config
    gc.collect()
    torch.cuda.empty_cache()

# 3. Final Summary
print("\n" + "="*70)
print("PHASE 2 RESULTS SUMMARY")
print("="*70)

if all_results:
    phase2_results_df = pd.concat(all_results, ignore_index=True)
    phase2_results_df.to_csv(f"{results_dir}/phase2_all_results.csv", index=False)

    cols = ['embedding_model', 'embedding_domain', 'MRR', 'Recall@10', 'NDCG@10']
    available_cols = [c for c in cols if c in phase2_results_df.columns]

    print(phase2_results_df[available_cols].sort_values('MRR', ascending=False).to_string(index=False))
else:
    print("❌ No results collected.")

### VI. Hybrid Retrieval (Dense + Sparse)

**Objective:**
Standard dense embeddings often fail on exact keyword matching (e.g., specific chemical names or citations). This phase tests whether a **Hybrid** approach—fusing dense semantic vectors with sparse lexical weights—improves retrieval on scientific text.

**Methodology:**
We utilize **BGE-M3**, a model capable of generating both dense embeddings and sparse (ColBERT-style) lexical weights, and **the victor of our dense-only embedding task** from above.
* **Mechanism:** We perform a weighted sum of normalized dense and sparse scores: $Score = \alpha \cdot S_{dense} + (1-\alpha) \cdot S_{sparse}$.
* **Variables:** We test $\alpha$ (Dense Weight) at **1.0** (Baseline), **0.7**, and **0.5**.

**Technical Note:**
To ensure correct serialization with Ray actors, we define the `_hybrid_preprocess` function to load the `BGEM3FlagModel` on demand within the worker process.

In [ ]:
from rapidfireai import Experiment
from rapidfireai.automl import RFLangChainRagSpec, RFvLLMModelConfig, RFGridSearch
from langchain_community.document_loaders import DirectoryLoader, JSONLoader
from langchain_huggingface import HuggingFaceEmbeddings
from datasets import Dataset
import pandas as pd
import numpy as np

# 1. Setup Phase 2b Directory
results_dir_2b = '/content/drive/MyDrive/RapidFire_QASPER_Results/phase2b'
os.makedirs(results_dir_2b, exist_ok=True)

print("="*70)
print("PHASE 2b: HYBRID RETRIEVAL (BGE-M3)")
print("="*70)

# 2. Data Preparation (Arrow-Free for Ray Safety)
# We re-load the dataset to ensure we have a clean dictionary-based structure
eval_set_path = Path("qasper_data") / "eval_set.json"
with open(eval_set_path, 'r') as f:
    eval_data = json.load(f)

# Sample 10% for speed
eval_df = pd.DataFrame(eval_data)
sampled_df = eval_df.sample(n=int(len(eval_df) * 0.1), random_state=42).reset_index(drop=True)
eval_dataset = Dataset.from_dict(sampled_df.to_dict(orient='list'))

print(f"✓ Eval dataset loaded: {len(eval_dataset)} samples")

# 3. Hybrid Logic Functions
def _hybrid_preprocess(batch: Dict[str, List], rag: RFLangChainRagSpec, dense_weight: float) -> Dict[str, List]:
    """
    Executes Hybrid Retrieval:
    1. Get candidate docs via standard retrieval.
    2. Load BGE-M3 model (on worker).
    3. Re-score candidates using Dense + Sparse fusion.
    """
    from FlagEmbedding import BGEM3FlagModel

    # Get initial candidates
    all_context = rag.get_context(batch_queries=batch["query"], serialize=False)

    # Load Scorer (Lazy load on GPU)
    model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True, device='cuda')

    reranked_context = []

    for query, docs in zip(batch["query"], all_context):
        if not docs:
            reranked_context.append([])
            continue

        doc_texts = [doc.page_content for doc in docs]

        # Encode Query & Docs (Dense + Sparse)
        q_out = model.encode([query], return_dense=True, return_sparse=True, return_colbert_vecs=False)
        d_out = model.encode(doc_texts, batch_size=4, return_dense=True, return_sparse=True, return_colbert_vecs=False)

        # Calculate Scores
        dense_scores = np.dot(d_out['dense_vecs'], q_out['dense_vecs'][0])

        # Sparse Dot Product
        q_lex = q_out['lexical_weights'][0]
        sparse_scores = []
        for d_lex in d_out['lexical_weights']:
            score = sum(q_lex.get(t, 0) * d_lex.get(t, 0) for t in set(q_lex) & set(d_lex))
            sparse_scores.append(score)
        sparse_scores = np.array(sparse_scores)

        # Normalize & Fuse
        if dense_scores.max() > 0: dense_scores /= dense_scores.max()
        if sparse_scores.max() > 0: sparse_scores /= sparse_scores.max()

        hybrid_scores = (dense_weight * dense_scores) + ((1 - dense_weight) * sparse_scores)

        # Sort by Hybrid Score
        ranked = sorted(zip(docs, hybrid_scores), key=lambda x: x[1], reverse=True)
        reranked_context.append([d for d, s in ranked])

    # Serialize
    serialized = rag.serialize_documents(reranked_context)
    retrieved_ids = [[d.metadata.get("doc_id", "") for d in docs] for docs in reranked_context]

    prompts = [[{"role": "user", "content": f"Context: {ctx[:500]}\nQ: {q}"}]
               for q, ctx in zip(batch["query"], serialized)]

    return {"prompts": prompts, "retrieved_documents": retrieved_ids, **batch}

# Wrappers for Grid Search
def preprocess_fn_dense(batch, rag, pm=None):
    # Standard retrieval (Dense weight = 1.0 implicitly)
    return _hybrid_preprocess(batch, rag, dense_weight=1.0)

def preprocess_fn_hybrid_07(batch, rag, pm=None): return _hybrid_preprocess(batch, rag, dense_weight=0.7)
def preprocess_fn_hybrid_05(batch, rag, pm=None): return _hybrid_preprocess(batch, rag, dense_weight=0.5)

def postprocess_fn(batch):
    # Ensure ground truth format matches metrics function expectations
    batch["ground_truth_documents"] = [[str(gid)] for gid in batch["ground_truth_id"]]
    return batch

# 4. Configuration & Execution
CONFIGS = [
    {"name": "dense_only_baseline", "fn": preprocess_fn_dense},
    {"name": "hybrid_0.7_dense", "fn": preprocess_fn_hybrid_07},
    {"name": "hybrid_0.5_dense", "fn": preprocess_fn_hybrid_05},
]

# RAG Config (Base)
rag_config = RFLangChainRagSpec(
    document_loader=DirectoryLoader(
        str(Path("qasper_data")), glob="corpus.jsonl", loader_cls=JSONLoader,
        loader_kwargs={"jq_schema": ".", "content_key": "text", "json_lines": True,
                       "metadata_func": lambda r, m: {"doc_id": r.get("_id"), "title": r.get("title", "")}}
    ),
    text_splitter=PHASE1_WINNER_SPLITTER,
    embedding_cls=HuggingFaceEmbeddings,
    embedding_kwargs={"model_name": "BAAI/bge-m3", "model_kwargs": {"device": "cuda"}, "encode_kwargs": {"batch_size": 4}},
    search_kwargs={"k": 20},
    enable_gpu_search=False # CPU Search for this phase to save GPU for the Model
)

vllm_config = RFvLLMModelConfig(
    model_config={"model": "Qwen/Qwen2.5-0.5B-Instruct", "dtype": "half", "gpu_memory_utilization": 0.2, "max_model_len": 1024, "disable_log_stats": True},
    sampling_params={"temperature": 0.1, "max_tokens": 16},
    rag=rag_config
)

# Run Loop
for cfg in CONFIGS:
    name = cfg["name"]
    save_path = f"{results_dir_2b}/{name}_results.csv"

    if os.path.exists(save_path):
        print(f"⏭️  Skipping {name} (Done)")
        continue

    print(f"🚀 Running: {name}")
    gc.collect(); torch.cuda.empty_cache()

    exp = Experiment(experiment_name=f"qasper_p2b_{name}", mode="evals")
    try:
        results = exp.run_evals(
            config_group=RFGridSearch({
                "vllm_config": vllm_config, "batch_size": 4,
                "preprocess_fn": cfg["fn"], "postprocess_fn": postprocess_fn,
                "compute_metrics_fn": compute_metrics_fn, "accumulate_metrics_fn": accumulate_metrics_fn
            }),
            dataset=eval_dataset, num_actors=1, num_shards=1, seed=42
        )

        # Save Result
        for _, metrics in results.values():
            mrr = metrics['MRR']['value']
            print(f"   ✅ MRR: {mrr:.4f}")
            pd.DataFrame([{'config': name, 'MRR': mrr}]).to_csv(save_path, index=False)

        exp.end()
    except Exception as e:
        print(f"❌ Error: {e}")

### VII. End Experiment

In [ ]:
try:
    experiment.end()
    print("✓ Phase 2 Global Experiment finalized and closed.")
except Exception as e:
    print(f"⚠️ Note: Experiment might have already ended: {e}")

# Cleanup
import gc
gc.collect()

## 3. Results Analysis & Visualization

This section analyzes the experimental results from Phases 1 and 2, generating publication-quality visualizations and statistical analyses. All metrics are logged to MLflow for reproducibility and exported as required artifacts.

### Phase 1: Chunking Strategy Analysis

**Research Question:** Does semantic-aware chunking (BM25-guided boundary detection) outperform fixed-size and structural chunking approaches for scientific document retrieval?

**Experimental Design:**
- **Independent Variables:**
  - Chunking Strategy: Fixed-256, Fixed-512, Structural, Semantic (BM25)
  - Overlap Type: None, Fixed (15%), Semantic
- **Dependent Variables:** MRR, Recall@10, NDCG@10
- **Total Configurations:** 12 (4 strategies × 3 overlap types)
- **Evaluation Set:** 317 queries (10% stratified sample of QASPER)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import mlflow
import json
import warnings
from google.colab import drive
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# Initialize MLflow
mlflow.set_experiment("QASPER_RAG_Optimization")

drive.mount('/content/drive')

# Load Phase 1 results
results_path = '/content/drive/MyDrive/RapidFire_QASPER_Results/phase1/all_results.csv'
phase1_df = pd.read_csv(results_path)

# Extract key metrics
metrics = ['MRR', 'Recall@10', 'NDCG@10']
results = phase1_df[['config_name'] + metrics].copy()

# Parse configuration names for analysis
def parse_config(name):
    """Extract chunking strategy and overlap type from config name."""
    parts = name.split('_')

    if parts[0] == 'fixed':
        chunking = f"Fixed-{parts[1]}"
    elif parts[0] == 'structural':
        chunking = "Structural"
    else:
        chunking = "Semantic (BM25)"

    if 'no_overlap' in name:
        overlap = "None"
    elif 'fixed_overlap' in name:
        overlap = "Fixed (15%)"
    elif 'semantic_overlap' in name:
        overlap = "Semantic"
    else:
        overlap = "None"

    return chunking, overlap

results[['Chunking Strategy', 'Overlap Type']] = results['config_name'].apply(
    lambda x: pd.Series(parse_config(x))
)

print("✓ Phase 1 Results Loaded")
print(f"  Configurations tested: {len(results)}")
print(f"  Metrics evaluated: {metrics}")

# Display summary table
print("\n" + "="*70)
print("PHASE 1 RESULTS SUMMARY (Sorted by MRR)")
print("="*70)
display_cols = ['config_name', 'MRR', 'Recall@10', 'NDCG@10']
print(results[display_cols].sort_values('MRR', ascending=False).to_string(index=False))

### I. Performance Comparison Across All Configurations

The horizontal bar chart below displays all 12 configurations ranked by each metric. Color coding indicates the chunking strategy, revealing clear performance clustering.

**Key Observations:**
- All **Semantic (BM25)** configurations (green) cluster at the top
- **Fixed-256** (blue) shows moderate performance
- **Fixed-512** and **Structural** approaches underperform consistently

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 6))

# Sort by MRR for consistent ordering
sorted_results = results.sort_values('MRR', ascending=True)

# Color mapping by chunking strategy
colors = {
    'Semantic (BM25)': '#2ecc71',  # Green - winner
    'Fixed-256': '#3498db',        # Blue
    'Fixed-512': '#9b59b6',        # Purple
    'Structural': '#e74c3c'        # Red
}

bar_colors = [colors[cs] for cs in sorted_results['Chunking Strategy']]

for idx, metric in enumerate(metrics):
    ax = axes[idx]
    bars = ax.barh(range(len(sorted_results)), sorted_results[metric],
                   color=bar_colors, edgecolor='white', linewidth=0.5)

    ax.set_yticks(range(len(sorted_results)))
    ax.set_yticklabels(sorted_results['config_name'].str.replace('_', '\n'), fontsize=8)
    ax.set_xlabel(metric, fontsize=12, fontweight='bold')

    # Add value labels
    for bar, val in zip(bars, sorted_results[metric]):
        ax.text(val + 0.005, bar.get_y() + bar.get_height()/2, f'{val:.3f}',
                va='center', fontsize=8)

axes[0].set_ylabel('Configuration', fontsize=12)

# Add legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=c, label=l) for l, c in colors.items()]
fig.legend(handles=legend_elements, loc='upper center', ncol=4,
           bbox_to_anchor=(0.5, 1.02), fontsize=10)

plt.suptitle('Phase 1: Chunking Strategy Performance Comparison',
             fontsize=14, fontweight='bold', y=1.06)
plt.tight_layout()

# Save for MLflow
fig.savefig('/content/phase1_metrics_comparison.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

print("✓ Figure saved: phase1_metrics_comparison.png")

### II. Aggregated Performance by Chunking Strategy

To isolate the effect of chunking strategy from overlap type, we average performance across all overlap configurations. This reveals the fundamental impact of how documents are segmented.

**Finding:** Semantic (BM25) chunking achieves **14.9% higher MRR** than the average of other strategies, with consistent gains across all metrics.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

# Aggregate by chunking strategy
chunking_avg = results.groupby('Chunking Strategy')[metrics].mean()
chunking_avg = chunking_avg.sort_values('MRR', ascending=False)

x = np.arange(len(chunking_avg))
width = 0.25

bars1 = ax.bar(x - width, chunking_avg['MRR'], width, label='MRR', color='#3498db')
bars2 = ax.bar(x, chunking_avg['Recall@10'], width, label='Recall@10', color='#2ecc71')
bars3 = ax.bar(x + width, chunking_avg['NDCG@10'], width, label='NDCG@10', color='#e74c3c')

ax.set_ylabel('Score', fontsize=12, fontweight='bold')
ax.set_xlabel('Chunking Strategy', fontsize=12, fontweight='bold')
ax.set_title('Average Performance by Chunking Strategy\n(Averaged Across All Overlap Types)',
             fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(chunking_avg.index, fontsize=11)
ax.legend(loc='upper right', fontsize=10)
ax.set_ylim(0, 0.75)

# Add value labels
for bars in [bars1, bars2, bars3]:
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.3f}', xy=(bar.get_x() + bar.get_width()/2, height),
                    xytext=(0, 3), textcoords="offset points", ha='center', fontsize=9)

plt.tight_layout()
fig.savefig('/content/phase1_chunking_comparison.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

# Print statistics
print("\n📊 Performance by Chunking Strategy:")
print(chunking_avg.round(4).to_string())

### III. Interaction Effects: Chunking × Overlap

The heatmap below visualizes the interaction between chunking strategy and overlap type. This factorial analysis reveals whether certain combinations produce synergistic effects.

**Key Insight:** The Semantic (BM25) row shows uniformly high performance (all cells > 0.21 MRR), indicating robustness to overlap choice. In contrast, other strategies show more variance.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

# Create pivot table
pivot_data = results.pivot_table(
    values='MRR',
    index='Chunking Strategy',
    columns='Overlap Type',
    aggfunc='mean'
)

# Reorder for visualization
chunk_order = ['Semantic (BM25)', 'Fixed-256', 'Fixed-512', 'Structural']
overlap_order = ['None', 'Fixed (15%)', 'Semantic']
pivot_data = pivot_data.reindex(index=chunk_order, columns=overlap_order)

sns.heatmap(pivot_data, annot=True, fmt='.4f', cmap='RdYlGn',
            linewidths=2, linecolor='white', cbar_kws={'label': 'MRR'},
            annot_kws={'fontsize': 12, 'fontweight': 'bold'}, ax=ax)

ax.set_title('MRR Heatmap: Chunking Strategy × Overlap Type', fontsize=14, fontweight='bold')
ax.set_xlabel('Overlap Type', fontsize=12, fontweight='bold')
ax.set_ylabel('Chunking Strategy', fontsize=12, fontweight='bold')

plt.tight_layout()
fig.savefig('/content/phase1_heatmap.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

### IV. Statistical Significance Analysis

To validate that the observed differences are meaningful (not due to random variation), we compute effect sizes and relative improvements.

In [ ]:
print("="*70)
print("STATISTICAL SIGNIFICANCE ANALYSIS")
print("="*70)

# Effect size: Semantic vs Others
semantic_mrr = results[results['Chunking Strategy'] == 'Semantic (BM25)']['MRR'].values
other_mrr = results[results['Chunking Strategy'] != 'Semantic (BM25)']['MRR'].values

# Cohen's d
pooled_std = np.sqrt((semantic_mrr.std()**2 + other_mrr.std()**2) / 2)
cohens_d = (semantic_mrr.mean() - other_mrr.mean()) / pooled_std

print(f"\n📊 Semantic (BM25) vs. All Other Strategies:")
print(f"   Semantic Mean MRR:     {semantic_mrr.mean():.4f}")
print(f"   Other Strategies MRR:  {other_mrr.mean():.4f}")
print(f"   Absolute Difference:   {semantic_mrr.mean() - other_mrr.mean():.4f}")
print(f"   Relative Improvement:  {(semantic_mrr.mean() - other_mrr.mean()) / other_mrr.mean() * 100:.1f}%")
print(f"\n📊 Effect Size (Cohen's d): {cohens_d:.2f}")

if cohens_d > 0.8:
    interpretation = "Large effect (d > 0.8)"
elif cohens_d > 0.5:
    interpretation = "Medium effect (0.5 < d < 0.8)"
else:
    interpretation = "Small effect (d < 0.5)"
print(f"   Interpretation: {interpretation}")

# Pairwise comparisons
print(f"\n📊 Pairwise Comparisons (vs. semantic_no_overlap):")
baseline = results[results['config_name'] == 'semantic_no_overlap']['MRR'].values[0]
print(f"   Baseline MRR: {baseline:.4f}")
print()

for _, row in results.sort_values('MRR', ascending=False).iterrows():
    if row['config_name'] != 'semantic_no_overlap':
        diff = baseline - row['MRR']
        pct = diff / row['MRR'] * 100 if row['MRR'] > 0 else 0
        direction = "+" if diff > 0 else ""
        print(f"   vs {row['config_name']:35s}: {direction}{pct:.1f}% ({diff:+.4f})")

### V. Winner Selection: Semantic BM25 (No Overlap)

Based on the comprehensive analysis above, we select **`semantic_no_overlap`** as the Phase 1 winner for the following reasons:

| Criterion | Value | Justification |
|-----------|-------|---------------|
| **MRR** | 0.2128 | 2nd highest overall; within 5% of best |
| **Recall@10** | 0.3060 | **Best** among all configurations |
| **NDCG@10** | 0.6253 | 3rd highest; competitive with top |
| **Simplicity** | ✓ | No overlap computation required |
| **Efficiency** | ✓ | Faster processing, smaller index |

**Rationale:** While `semantic_fixed_overlap` achieves marginally higher MRR (0.2229 vs 0.2128), the `semantic_no_overlap` configuration:
1. Achieves the **best Recall@10**, which is critical for RAG pipelines where missing the relevant document is costly
2. Requires **no additional overlap computation**, reducing complexity
3. Produces **fewer, more focused chunks**, leading to more efficient indexing and retrieval

For Phase 2, we "lock in" the Semantic BM25 chunker with no overlap.

In [ ]:
# Log Phase 1 results to MLflow
with mlflow.start_run(run_name="Phase1_Chunking_Analysis"):

    # Log best configuration metrics
    best_config = results[results['config_name'] == 'semantic_no_overlap'].iloc[0]
    mlflow.log_param("winner_config", "semantic_no_overlap")
    mlflow.log_param("chunking_strategy", "Semantic (BM25)")
    mlflow.log_param("overlap_type", "None")
    mlflow.log_param("total_configs_tested", len(results))

    mlflow.log_metric("winner_MRR", best_config['MRR'])
    mlflow.log_metric("winner_Recall_at10", best_config['Recall@10'])
    mlflow.log_metric("winner_NDCG_at10", best_config['NDCG@10'])
    mlflow.log_metric("cohens_d_effect_size", cohens_d)
    mlflow.log_metric("improvement_over_baseline_pct",
                      (semantic_mrr.mean() - other_mrr.mean()) / other_mrr.mean() * 100)

    # Log artifacts (figures)
    mlflow.log_artifact('/content/phase1_metrics_comparison.png')
    mlflow.log_artifact('/content/phase1_chunking_comparison.png')
    mlflow.log_artifact('/content/phase1_heatmap.png')

    # Log results CSV
    results.to_csv('/content/phase1_results_processed.csv', index=False)
    mlflow.log_artifact('/content/phase1_results_processed.csv')

    print("✓ Phase 1 results logged to MLflow")
    print(f"  Run ID: {mlflow.active_run().info.run_id}")

---

## Phase 1 Conclusion

**Key Findings:**
1. **Semantic (BM25) chunking dominates** across all metrics with a large effect size (Cohen's d = 2.59)
2. **Overlap type has minimal impact** when using semantic chunking (all variants score > 0.21 MRR)
3. **Fixed-size chunking underperforms**, particularly at larger chunk sizes (512 tokens)
4. The **simpler no-overlap configuration** achieves the best Recall@10, making it ideal for RAG

**Winner Locked In:** `SemanticBM25Splitter(similarity_threshold=0.3, min_chunk_sentences=3)` with no overlap

This configuration will be used as the foundation for Phase 2 embedding optimization.

---

## Phase 2: Embedding Model Optimization

**Research Question:** Do modern retrieval-optimized embedding models (2024) significantly outperform general-purpose 2021 baselines on scientific document retrieval?

**Experimental Design:**
- **Fixed Variable:** Semantic BM25 chunking (Phase 1 winner)
- **Independent Variable:** Embedding model (5 models across 4 domains)
- **Dependent Variables:** MRR, Recall@10, NDCG@10

| Model | Era | Domain | Dimensions |
|-------|-----|--------|------------|
| all-MiniLM-L6-v2 | 2021 | General | 384 |
| all-mpnet-base-v2 | 2021 | General | 768 |
| allenai-specter | 2020 | Scientific | 768 |
| multi-qa-mpnet-base | 2021 | QA-Tuned | 768 |
| **bge-m3** | **2024** | **Retrieval** | **1024** |

In [ ]:
# Load Phase 2 results
phase2_path = '/content/drive/MyDrive/RapidFire_QASPER_Results/phase2/phase2_all_results.csv'
phase2_df = pd.read_csv(phase2_path)

# Extract key columns
phase2 = phase2_df[['embedding_model', 'embedding_domain', 'MRR', 'Recall@10', 'NDCG@10']].copy()

# Load Phase 2b (Hybrid) results
phase2b_dir = '/content/drive/MyDrive/RapidFire_QASPER_Results/phase2b'
phase2b_files = [
    f'{phase2b_dir}/dense_only_baseline_results.csv',
    f'{phase2b_dir}/hybrid_0.7_dense_results.csv',
    f'{phase2b_dir}/hybrid_0.5_dense_results.csv',
]
phase2b = pd.concat([pd.read_csv(f) for f in phase2b_files], ignore_index=True)

print("="*70)
print("PHASE 2 DATA LOADED")
print("="*70)
print(f"\nPhase 2 (Embedding Models): {len(phase2)} configurations")
print(phase2[['embedding_model', 'MRR', 'Recall@10', 'NDCG@10']].sort_values('MRR', ascending=False).to_string(index=False))

print(f"\nPhase 2b (Hybrid Retrieval): {len(phase2b)} configurations")
print(phase2b[['config', 'MRR', 'Recall@10', 'NDCG@10']].to_string(index=False))

### I. Embedding Model Performance Comparison

The chart below compares MRR across all tested embedding models. The dashed line indicates the Phase 1 baseline (MiniLM with semantic chunking).

**Key Finding:** BGE-M3 (2024 SOTA) achieves **16.8% higher MRR** than the 2021 baseline, validating the hypothesis that modern retrieval-optimized models significantly outperform general-purpose encoders.

**Surprising Result:** The domain-specific `allenai-specter` model, trained on scientific papers, performs *worst* — suggesting that retrieval-specific training objectives matter more than domain matching.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

# Sort by MRR
phase2_sorted = phase2.sort_values('MRR', ascending=True)

# Color by domain
colors_map = {
    'retrieval-2024': '#2ecc71',  # Green - modern SOTA
    'general': '#3498db',          # Blue - baseline
    'qa-tuned': '#9b59b6',         # Purple
    'scientific': '#e74c3c',       # Red
}
bar_colors = [colors_map.get(d, '#95a5a6') for d in phase2_sorted['embedding_domain']]

y_pos = np.arange(len(phase2_sorted))
bars = ax.barh(y_pos, phase2_sorted['MRR'], color=bar_colors, edgecolor='white', height=0.6)

ax.set_yticks(y_pos)
ax.set_yticklabels(phase2_sorted['embedding_model'], fontsize=11)
ax.set_xlabel('MRR', fontsize=12, fontweight='bold')
ax.set_title('Phase 2: Embedding Model Performance\n(Using Semantic BM25 Chunking from Phase 1)',
             fontsize=14, fontweight='bold')

# Value labels
for bar, val in zip(bars, phase2_sorted['MRR']):
    ax.text(val + 0.005, bar.get_y() + bar.get_height()/2, f'{val:.4f}',
            va='center', fontsize=10, fontweight='bold')

# Baseline reference line
baseline_mrr = phase2[phase2['embedding_model'] == 'all-MiniLM-L6-v2']['MRR'].values[0]
ax.axvline(x=baseline_mrr, color='gray', linestyle='--', alpha=0.7)

# Legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#2ecc71', label='2024 SOTA'),
    Patch(facecolor='#3498db', label='General (2021)'),
    Patch(facecolor='#9b59b6', label='QA-Tuned'),
    Patch(facecolor='#e74c3c', label='Scientific'),
]
ax.legend(handles=legend_elements, loc='lower right', fontsize=9)

plt.tight_layout()
fig.savefig('/content/phase2_embedding_comparison.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

### II. Hybrid Retrieval: Dense + Sparse Fusion (Phase 2b)

**Hypothesis:** Scientific text contains domain-specific terminology (chemical names, citations, acronyms) that benefits from exact lexical matching. A hybrid approach combining dense semantic vectors with sparse lexical weights should outperform pure dense retrieval.

**Method:** Using BGE-M3's multi-vector output, we compute:
$$Score = \alpha \cdot S_{dense} + (1-\alpha) \cdot S_{sparse}$$

Where α is the dense weight (1.0 = pure dense, 0.5 = equal weighting).

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 5))

metrics = ['MRR', 'Recall@10', 'NDCG@10']
colors = ['#3498db', '#2ecc71', '#e74c3c']

# Create readable labels
phase2b['label'] = phase2b['config'].map({
    'dense_only_baseline': 'Dense Only\n(α=1.0)',
    'hybrid_0.7_dense': 'Hybrid\n(α=0.7)',
    'hybrid_0.5_dense': 'Hybrid\n(α=0.5)'
})

# Sort: Dense first, then hybrids
order = ['dense_only_baseline', 'hybrid_0.7_dense', 'hybrid_0.5_dense']
phase2b_sorted = phase2b.set_index('config').loc[order].reset_index()

for idx, metric in enumerate(metrics):
    ax = axes[idx]
    bars = ax.bar(phase2b_sorted['label'], phase2b_sorted[metric],
                  color=colors[idx], edgecolor='white', width=0.6)

    ax.set_ylabel(metric if idx == 0 else '', fontsize=12, fontweight='bold')
    ax.set_title(metric, fontsize=14, fontweight='bold')

    # Value labels
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.4f}', xy=(bar.get_x() + bar.get_width()/2, height),
                    xytext=(0, 3), textcoords="offset points", ha='center', fontsize=10, fontweight='bold')

    # Adjust y-axis
    ymin = phase2b_sorted[metric].min() * 0.95
    ymax = phase2b_sorted[metric].max() * 1.03
    ax.set_ylim(ymin, ymax)

plt.suptitle('Phase 2b: Hybrid Retrieval (Dense + Sparse Fusion)\nScore = α·Dense + (1-α)·Sparse',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
fig.savefig('/content/phase2b_hybrid_comparison.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

# Print improvement
dense_mrr = phase2b[phase2b['config'] == 'dense_only_baseline']['MRR'].values[0]
hybrid_mrr = phase2b[phase2b['config'] == 'hybrid_0.5_dense']['MRR'].values[0]
print(f"\n📊 Hybrid Improvement: {(hybrid_mrr - dense_mrr) / dense_mrr * 100:.1f}% MRR gain from adding sparse retrieval")

### III. Cumulative Improvement: Full Experimental Journey

The waterfall chart below shows the progressive MRR improvements from our worst baseline configuration to the final optimized system.

| Phase | Configuration | MRR | Cumulative Gain |
|-------|--------------|-----|-----------------|
| Baseline | Fixed-512, No Overlap | 0.1819 | — |
| Phase 1 | Semantic BM25 | 0.2128 | +17.0% |
| Phase 2 | BGE-M3 Dense | 0.2486 | +36.7% |
| Phase 2b | Hybrid 50/50 | **0.2770** | **+52.3%** |

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

# Define progression
stages = [
    ('Phase 1 Baseline\n(Fixed-512, No Overlap)', 0.1819),
    ('Phase 1 Winner\n(Semantic BM25)', 0.2128),
    ('Phase 2 Winner\n(BGE-M3 Dense)', 0.2486),
    ('Phase 2b Winner\n(Hybrid 50/50)', 0.2770),
]

labels = [s[0] for s in stages]
values = [s[1] for s in stages]

# Colors
colors = ['#95a5a6'] + ['#2ecc71'] * 3

x = np.arange(len(labels))
bars = ax.bar(x, values, color=colors, edgecolor='white', width=0.6)

# Labels
for i, (bar, val) in enumerate(zip(bars, values)):
    ax.text(bar.get_x() + bar.get_width()/2, val + 0.005, f'{val:.4f}',
            ha='center', fontsize=11, fontweight='bold')
    if i > 0:
        pct_imp = (val - values[0]) / values[0] * 100
        ax.text(bar.get_x() + bar.get_width()/2, val - 0.015, f'+{pct_imp:.1f}%',
                ha='center', fontsize=9, color='white', fontweight='bold')

ax.set_xticks(x)
ax.set_xticklabels(labels, fontsize=10)
ax.set_ylabel('MRR', fontsize=12, fontweight='bold')
ax.set_title('Cumulative MRR Improvement Across Experimental Phases', fontsize=14, fontweight='bold')
ax.set_ylim(0, 0.35)

# Total improvement annotation
total_imp = (values[-1] - values[0]) / values[0] * 100
ax.annotate(f'Total Improvement: +{total_imp:.1f}%',
            xy=(3, values[-1]), xytext=(2.2, 0.32),
            fontsize=12, fontweight='bold', color='#2ecc71',
            arrowprops=dict(arrowstyle='->', color='#2ecc71', lw=2))

plt.tight_layout()
fig.savefig('/content/phase2_cumulative_improvement.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

### IV. Complete Results Overview

The heatmap below consolidates all experimental configurations, sorted by MRR. This provides a single view of the entire optimization landscape.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

# Build combined dataframe
all_results = []

# Phase 1 winner
all_results.append({
    'Configuration': 'P1: Semantic BM25 (No Overlap)',
    'MRR': 0.2128, 'Recall@10': 0.3060, 'NDCG@10': 0.6253
})

# Phase 2 models
for _, row in phase2.iterrows():
    all_results.append({
        'Configuration': f"P2: {row['embedding_model']}",
        'MRR': row['MRR'], 'Recall@10': row['Recall@10'], 'NDCG@10': row['NDCG@10']
    })

# Phase 2b hybrid
for _, row in phase2b.iterrows():
    label = row['config'].replace('_', ' ').title()
    all_results.append({
        'Configuration': f"P2b: {label}",
        'MRR': row['MRR'], 'Recall@10': row['Recall@10'], 'NDCG@10': row['NDCG@10']
    })

combined_df = pd.DataFrame(all_results)
heatmap_data = combined_df.set_index('Configuration')[['MRR', 'Recall@10', 'NDCG@10']]
heatmap_data = heatmap_data.sort_values('MRR', ascending=False)

sns.heatmap(heatmap_data, annot=True, fmt='.4f', cmap='RdYlGn',
            linewidths=1, linecolor='white', cbar_kws={'label': 'Score'},
            annot_kws={'fontsize': 9}, ax=ax)

ax.set_title('All Experimental Configurations: Performance Heatmap', fontsize=14, fontweight='bold')
ax.set_xlabel('Metric', fontsize=12, fontweight='bold')
ax.set_ylabel('Configuration', fontsize=12, fontweight='bold')

plt.tight_layout()
fig.savefig('/content/phase2_combined_heatmap.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

In [ ]:
# Log Phase 2 results to MLflow
with mlflow.start_run(run_name="Phase2_Complete_Analysis"):

    # Phase 2 winner
    mlflow.log_param("phase2_winner", "bge-m3")
    mlflow.log_metric("phase2_winner_MRR", 0.2486)

    # Phase 2b winner
    mlflow.log_param("phase2b_winner", "hybrid_0.5_dense")
    mlflow.log_metric("phase2b_winner_MRR", 0.2770)
    mlflow.log_metric("phase2b_winner_Recall_at10", 0.3754)
    mlflow.log_metric("phase2b_winner_NDCG_at10", 0.7913)

    # Total improvement
    mlflow.log_metric("total_improvement_pct", 52.3)
    mlflow.log_metric("baseline_MRR", 0.1819)
    mlflow.log_metric("final_MRR", 0.2770)

    # Log artifacts
    mlflow.log_artifact('/content/phase2_embedding_comparison.png')
    mlflow.log_artifact('/content/phase2b_hybrid_comparison.png')
    mlflow.log_artifact('/content/phase2_cumulative_improvement.png')
    mlflow.log_artifact('/content/phase2_combined_heatmap.png')

    print("✓ Phase 2 results logged to MLflow")
    print(f"  Run ID: {mlflow.active_run().info.run_id}")

---

## Experimental Conclusions

### Summary of Findings

| Phase | Key Finding | Impact |
|-------|-------------|--------|
| **Phase 1** | Semantic BM25 chunking outperforms fixed-size approaches | +17% MRR |
| **Phase 2** | Modern 2024 embeddings (BGE-M3) beat 2021 baselines | +17% MRR |
| **Phase 2b** | Hybrid dense+sparse retrieval captures lexical matches | +14% MRR |
| **Total** | Cumulative optimization from baseline to best | **+52% MRR** |

### Final Optimized Configuration

**Chunking:**   SemanticBM25Splitter(threshold=0.3, min_sentences=3)

**Overlap:**    None

**Embedding:**  BAAI/bge-m3 (1024-dim, 8K context)

**Retrieval:**  Hybrid (α=0.5 dense + 0.5 sparse)